In [1]:
import pandas as pd

In [2]:
df =pd.read_csv("heart.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [8]:
df1 = pd.read_excel("LCDataDictionary.xlsx")
df1.shape

(153, 2)

KeyboardInterrupt: 

284

In [ ]:
# Step 0: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, classification_report
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline


# Step 1: Load Data
df = pd.read_csv("heart.csv")  # Replace with the path if needed
print(df.head())

# Step 2: EDA
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title("Feature Correlation Heatmap")
plt.show()

# Check target distribution
sns.countplot(x='target', data=df)
plt.title("Target Class Distribution")
plt.show()

# Boxplot example
sns.boxplot(x='target', y='age', data=df)
plt.title("Age vs Target")
plt.show()

# Step 3: Feature Selection & Scaling
X = df.drop('target', axis=1)
y = df['target']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Train Models
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Gradient Boosting
gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)

# SVM
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)

# Neural Network
nn_model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500)
nn_model.fit(X_train, y_train)
nn_pred = nn_model.predict(X_test)

# Step 5: Evaluate Models
models = {'Gradient Boosting': gb_model, 'SVM': svm_model, 'Neural Net': nn_model}
for name, model in models.items():
    preds = model.predict(X_test)
    probs = model.predict_proba(X_test)[:, 1]
    print(f"\n{name}")
    print("F1 Score:", f1_score(y_test, preds))
    print("AUC-ROC:", roc_auc_score(y_test, probs))
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, probs)
    plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc_score(y_test, probs):.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("ROC Curves")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid()
plt.show()

# Step 6: Feature Importance from Gradient Boosting
feat_importance = pd.Series(gb_model.feature_importances_, index=df.columns[:-1])
feat_importance.nlargest(10).plot(kind='barh')
plt.title("Top 10 Important Features")
plt.show()
